In [6]:
import numpy as np
import pandas as pd
from numpy import *

#Variables that will be averaged after 5 experiments
ultimate_L_s = 0
ultimate_L_q = 0
ultimate_W_q = 0
ultimate_W_s = 0
ultimate_c = 0
random.seed(123)
z =1 #experiment number
while z <6:
    serv_rate=5 
    poiss_rate=3

   
    serv_times = np.random.exponential(1/serv_rate,10000) 

    arriv_times_demo = np.random.exponential(1/poiss_rate,10000)
    arriv_times = []
    for i in range(0,len(serv_times)):
        arriv_times.append(sum(arriv_times_demo[0:i+1]))


    depart_times = [serv_times[0]+arriv_times[0]]

    #Below part is conducting the arrival and departure times of the customers. First checks if the previous customer 
    # is still in the system or not and then accordingly determining the current customer's departure time.
    for i in range(1,len(serv_times)):
        if arriv_times[i] < depart_times[i-1]:
            depart_times.append(depart_times[i-1]+serv_times[i])
        else:
            depart_times.append(arriv_times[i]+serv_times[i])

    # In below determining the time each customer waited in the queue and the system using the data of arrival, service and departure time
    queue_times = [max(0,depart_times[i]-arriv_times[i]-serv_times[i]) for i in range(0,len(serv_times))]
    syst_times = [depart_times[i]-arriv_times[i] for i in range(0,len(serv_times))]

    #Below I rounded all the numbers because there were extremely long decimals (probably sth. about the numpy)
    serv_times = np.round_(serv_times,decimals=3)
    arriv_times = np.round_(arriv_times,decimals=3)
    depart_times = np.round_(depart_times,decimals=3)
    queue_times = np.round_(queue_times,decimals=3)
    syst_times = np.round_(syst_times,decimals=3)

    df = pd.DataFrame(list(zip(arriv_times, serv_times, depart_times, syst_times, queue_times)),columns=["Arrival Times", "Service Times", "Departure Times", "Total Time Spent in System", "Time Spent in Queue" ],index= range(1,len(serv_times)+1))
    if z ==1:
        print(df.head(10))




    t_i = {} #t_i represents the total time there were i people in the system
    p_i = {} #p_i represents the prob of having i people at an any given time in the system



    # Below is the algorithm I used to determine each t_i. I won't get in to very detail since it would take paragraphs to explain it.
    # Briefly what my strategy is iterating over the every customer's departure time and determining how many next customers arrived
    # before it left. I would happily get in detail if you furtherly ask me.
    for i in range(1,len(serv_times)):

        k = 0
        while i+1+k < len(serv_times) and arriv_times[i+1+k] < depart_times[i] :
            k += 1

        m=k
        while m >= 0 :
            key = "t_" + str(m+1)
            added = depart_times[i]-arriv_times[i+m]

            if added > depart_times[i]-depart_times[i-1]:
                added = depart_times[i]-depart_times[i-1]

            if m < k:
                added -= depart_times[i]-arriv_times[i+m+1]
                if depart_times[i]-arriv_times[i+m+1] > depart_times[i]-depart_times[i-1]:
                    added = 0

            if key in t_i.keys():
                t_i[key] += added


            else:
                t_i[key] = added
            m -= 1




    #below is the part to determine p_i's accordingly with t_i's
    sum_ = 0
    for i in t_i.values():
        sum_ = sum_ + i

    t_i["t_0"]= depart_times[-1] - sum_
    sum_ += t_i["t_0"]


    for every in t_i.keys():
        index = "p" + every[-2:]
        p_i[index] = t_i[every]/sum_



    L_s = 0
    L_q = 0
    W_q = sum(queue_times)/len(serv_times)
    W_s = sum(syst_times)/len(serv_times)
    c = (depart_times[-1]-t_i["t_0"])/depart_times[-1]


    # Below is the part finding the L_s's and L_q's using the p_i data
    for every in t_i.keys():
        index=every[-2:]
        intg= int(index[-1:])
        L_s += intg*p_i["p" + index]
        L_q += max(0,(intg-1))*p_i["p" + index]
        
    ultimate_L_s +=  L_s 
    ultimate_L_q += L_q
    ultimate_W_q +=  W_q
    ultimate_W_s += W_s
    ultimate_c += c
    z+=1
        
        
        

    # Theorical (calculated) values of the variables
rho = (poiss_rate/serv_rate)
thrt_Ls = rho/(1-rho)
thrt_Lq = rho*rho/(1-rho)
thrt_Ws = thrt_Ls/poiss_rate
thrt_Wq = rho/(serv_rate*(1-rho))
thrt_c = rho

    #Comparing the theoricaland the calculated variables, determining accuracies
L_s_ = [ultimate_L_s/5, thrt_Ls, "% " + str(np.round_(100 - abs(100*((thrt_Ls-(ultimate_L_s/5))/thrt_Ls)),decimals=3))]
L_q_ = [ultimate_L_q/5, thrt_Lq, "% " + str(np.round_(100 - abs(100*((thrt_Lq- (ultimate_L_q/5))/thrt_Lq)),decimals=3))]
W_s_ = [ultimate_W_s/5, thrt_Ws, "% " + str(np.round_(100 - abs(100*((thrt_Ws-(ultimate_W_s/5))/thrt_Ws)),decimals=3))]
W_q_ = [ultimate_W_q/5, thrt_Wq, "% " + str(np.round_(100 - abs(100*(thrt_Wq-(ultimate_W_q/5))/thrt_Wq),decimals=3))]
c_ = [ultimate_c/5, thrt_c, "% " + str(np.round_(100 - abs(100*(thrt_c-(ultimate_c/5))/thrt_c) ,decimals=3))]

df2 = pd.DataFrame(list(zip(L_s_, L_q_ , W_s_ , W_q_, c_)),columns=["Ls", "Lq", "Ws", "Wq", "c" ],index= ["By Simulation", "By Calculation", "Accuracy"])
        
print(df2)
   




    Arrival Times  Service Times  Departure Times  Total Time Spent in System  \
1           0.363          0.238            0.601                       0.238   
2           0.973          0.067            1.041                       0.067   
3           1.552          0.051            1.603                       0.051   
4           1.704          0.160            1.865                       0.160   
5           1.769          0.254            2.119                       0.350   
6           1.809          0.110            2.229                       0.420   
7           2.186          0.790            3.019                       0.833   
8           3.223          0.231            3.454                       0.231   
9           4.332          0.131            4.463                       0.131   
10          4.477          0.100            4.577                       0.100   

    Time Spent in Queue  
1                 0.000  
2                 0.000  
3                 0.000  
4   